In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from keras.preprocessing.image import ImageDataGenerator
import keras

2024-04-29 05:53:18.749415: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-29 05:53:18.791069: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX512F AVX512_VNNI, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model = keras.models.load_model('Models/model_9929.keras')

model.summary()

2024-04-29 05:53:20.216399: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-29 05:53:20.252087: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-29 05:53:20.255870: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1000)              25636712  
                                                                 
 dense_4 (Dense)             (None, 1)                 1001      
                                                                 
Total params: 25637713 (97.80 MB)
Trainable params: 25584593 (97.60 MB)
Non-trainable params: 53120 (207.50 KB)
_________________________________________________________________


In [3]:
df = pd.read_hdf('../../data/CNN Disaster/test_verify_data.h5', 'df')
df

,image,width,height,size
0,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",124,124,15376
1,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",124,124,15376
2,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",124,124,15376
3,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",124,124,15376
4,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",124,124,15376
...,...,...,...,...
3842,"[[[9, 15, 11], [9, 14, 11], [9, 14, 11], [9, 1...",124,124,15376
3843,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",124,124,15376
3844,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",124,124,15376
3845,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",124,124,15376


### Resize the test image dataset

Resizing the test dataset and normalizing the values before it gets inserted into the model

In [4]:
import concurrent.futures
import numpy as np
from skimage.transform import resize

def resize_image(image):
    return resize(image, (224, 224), anti_aliasing=True, preserve_range=True).astype(image.dtype)

def process_images(image_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        return list(executor.map(resize_image, image_list))

imgs = process_images(df["image"].values)

In [8]:
test_float_images = []
for image in imgs:
    float_image = np.array(image).astype(np.float32)
    test_float_images.append(float_image)

test_float_images = np.array(test_float_images)/255

### Predict values and output to CSV file

In [9]:
res = model.predict(test_float_images).round()

2024-04-29 05:54:50.299027: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2024-04-29 05:54:50.598308: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


121/121 [==============================] - 5s 32ms/step


In [10]:
df["pred"] = res
df["pred"] = df["pred"].astype(int)

In [11]:
df[["pred"]].to_csv("test_images_flooding-fire_predictions.csv")

### Evaluate validation set accuracy and recall on damage level

In [12]:
validation_dataset = np.load('../../data/CNN Disaster/test_data.npz', allow_pickle=True)

In [13]:
validation_df = pd.DataFrame()
validation_df["label"] = validation_dataset["labels"]
validation_df["damage_label"] = validation_dataset["damage_labels"]
validation_df["predicted"] = model.predict(validation_dataset["images"]).round()

validation_df

88/88 [==============================] - 3s 34ms/step


,label,damage_label,predicted
0,1.0,0,1.0
1,1.0,0,1.0
2,0.0,0,0.0
3,0.0,0,0.0
4,1.0,0,1.0
...,...,...,...
2797,0.0,0,0.0
2798,0.0,0,0.0
2799,1.0,0,1.0
2800,1.0,3,1.0


In [14]:
socal = validation_df[validation_df["label"] == 1]
midwest = validation_df[validation_df["label"] == 0]

In [15]:
socal_res = []
for name, group in socal.groupby("damage_label"):
    TP = sum((group['predicted'] == 1) & (group['label'] == 1))
    FN = sum((group['predicted'] != 1) & (group['label'] == 1))
    TN = sum((group['predicted'] != 1) & (group['label'] != 1))
    FP = sum((group['predicted'] == 1) & (group['label'] != 1))
    
    recall = TP / (TP + FN)
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    
    socal_res.append({
        'damage_label': name,
        'recall': f"{recall:.4%}",
        'accuracy': f"{accuracy:.4%}",
        'count': len(group)
    })

print("Socal predictions")
pd.DataFrame(socal_res)

Socal predictions


,damage_label,recall,accuracy,count
0,0,99.7628%,99.7628%,1265
1,1,80.0000%,80.0000%,10
2,2,100.0000%,100.0000%,9
3,3,99.4413%,99.4413%,179


In [16]:
midwest_res = []
for name, group in midwest.groupby("damage_label"):
    TP = sum((group['predicted'] == 0) & (group['label'] == 0))
    FN = sum((group['predicted'] != 0) & (group['label'] == 0))
    TN = sum((group['predicted'] != 0) & (group['label'] != 0))
    FP = sum((group['predicted'] == 0) & (group['label'] != 0))
    
    recall = TP / (TP + FN)
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    
    midwest_res.append({
        'damage_label': name,
        'recall': f"{recall:.4%}",
        'accuracy': f"{accuracy:.4%}",
        'count': len(group)
    })

print("Midwest predictions")
pd.DataFrame(midwest_res)

Midwest predictions


,damage_label,recall,accuracy,count
0,0,99.4582%,99.4582%,1292
1,1,100.0000%,100.0000%,18
2,2,100.0000%,100.0000%,14
3,3,100.0000%,100.0000%,15
